In [1]:
from datasets import load_dataset

dataset = load_dataset("rungalileo/20_Newsgroups_Fixed")

Found cached dataset csv (/home/rajabyfa/.cache/huggingface/datasets/rungalileo___csv/rungalileo--20_Newsgroups_Fixed-edf414ecc72dd622/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def trim_dataset(dataset, key):
    labels = {}
    for item in dataset[key]:
        if item['label'] not in labels:
            labels[item['label']] = 0
        labels[item['label']] += 1
    print(labels)
    before_len = len(dataset[key])
    print(before_len)
    dataset[key] = dataset[key].filter(lambda example: example['label'] and example['label'] != "None" and example['text'] and len(example["text"]) >= 10)
    after_len = len(dataset[key])
    print(after_len)
    print("the number of removed items : ", before_len - after_len)
    labels = {}
    for item in dataset[key]:
        if item['label'] not in labels:
            labels[item['label']] = 0
        labels[item['label']] += 1
    print(labels)
    return dataset

In [3]:
dataset = trim_dataset(dataset, "train")
dataset = trim_dataset(dataset, "test")

Loading cached processed dataset at /home/rajabyfa/.cache/huggingface/datasets/rungalileo___csv/rungalileo--20_Newsgroups_Fixed-edf414ecc72dd622/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-22327e0021383721.arrow


{'rec.autos': 539, 'comp.sys.mac.hardware': 537, 'comp.graphics': 551, 'sci.space': 564, 'talk.politics.guns': 526, 'sci.med': 572, 'comp.sys.ibm.pc.hardware': 562, 'comp.os.ms-windows.misc': 555, 'rec.motorcycles': 551, 'talk.religion.misc': 339, None: 650, 'misc.forsale': 565, 'alt.atheism': 449, 'sci.electronics': 563, 'comp.windows.x': 576, 'rec.sport.hockey': 559, 'rec.sport.baseball': 547, 'soc.religion.christian': 582, 'talk.politics.mideast': 521, 'talk.politics.misc': 438, 'sci.crypt': 568}
11314
10664
the number of removed items :  650
{'rec.autos': 539, 'comp.sys.mac.hardware': 537, 'comp.graphics': 551, 'sci.space': 564, 'talk.politics.guns': 526, 'sci.med': 572, 'comp.sys.ibm.pc.hardware': 562, 'comp.os.ms-windows.misc': 555, 'rec.motorcycles': 551, 'talk.religion.misc': 339, 'misc.forsale': 565, 'alt.atheism': 449, 'sci.electronics': 563, 'comp.windows.x': 576, 'rec.sport.hockey': 559, 'rec.sport.baseball': 547, 'soc.religion.christian': 582, 'talk.politics.mideast': 521,

Loading cached processed dataset at /home/rajabyfa/.cache/huggingface/datasets/rungalileo___csv/rungalileo--20_Newsgroups_Fixed-edf414ecc72dd622/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-ea989db5d94eb33f.arrow


7019
the number of removed items :  513
{'rec.autos': 354, 'comp.windows.x': 370, 'talk.politics.mideast': 353, 'talk.religion.misc': 230, 'sci.med': 371, 'soc.religion.christian': 380, 'comp.graphics': 370, 'comp.os.ms-windows.misc': 369, 'rec.motorcycles': 361, 'alt.atheism': 290, 'comp.sys.mac.hardware': 361, 'misc.forsale': 368, 'talk.politics.guns': 337, 'sci.space': 368, 'comp.sys.ibm.pc.hardware': 374, 'sci.crypt': 360, 'rec.sport.baseball': 366, 'rec.sport.hockey': 378, 'talk.politics.misc': 292, 'sci.electronics': 367}


In [4]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

from datasets import load_dataset

# First make the kfold object
folds = StratifiedKFold(n_splits=5)

# Now make our splits based off of the labels. 
# We can use `np.zeros()` here since it only works off of indices, we really care about the labels
splits = folds.split(np.zeros(dataset["train"].num_rows), dataset["train"]["label"])


In [5]:
import copy
# dataset2 = copy.deepcopy(dataset1)
new_datasets = [copy.deepcopy(dataset) for i in range(5)]
i = 0
for train_ids, val_ids in splits:
    new_datasets[i]["validation"] = new_datasets[i]["train"].select(val_ids)
    new_datasets[i]["train"] = new_datasets[i]["train"].select(train_ids)
    i += 1

In [6]:
for i in range(5):
    print(len(new_datasets[i]['train']))
    print(len(new_datasets[i]['validation']))

8531
2133
8531
2133
8531
2133
8531
2133
8532
2132


In [7]:
from transformers import RobertaTokenizer, BertTokenizer

# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [8]:
# comp.graphics
# comp.os.ms-windows.misc
# comp.sys.ibm.pc.hardware
# comp.sys.mac.hardware
# comp.windows.x
# rec.autos
# rec.motorcycles
# rec.sport.baseball
# rec.sport.hockey
# sci.crypt
# <sci.electronics
# sci.med
# sci.space
# misc.forsale
# talk.politics.misc
# talk.politics.guns
# talk.politics.mideast
# talk.religion.misc
# alt.atheism
# soc.religion.christian
# None

import torch


def collate_label_set(data):
    level1 = [
        "comp.os", "comp.sys", "comp.windows", "comp.graphics", "rec.motorcycles", "rec.sport", "rec.autos", "talk.religion",
        "sci.electronics", "sci.med", "sci.space", "misc.forsale", "talk.politics", "sci.crypt", "alt.atheism", "soc.religion"
    ]
    level2_pass = {
        "comp.windows", "comp.os", "talk.religion", "soc.religion"
    }
    level2 = [
        "misc", "guns", "ibm", "mac", "baseball", "hockey", "mideast", "None"
    ]
    for item in data:
        if item['text'] is None:
            item['text'] = " "
        try:
            label = item['label']
            if not label:
                label = "None"
                item['level1'] = level1.index(label)
                item['level2'] = level2.index("None")
            else:
                label = label.split(".")
                label_two = ".".join(label[:2])
                if label_two not in level1:
                    print(label[0])
                else:
                    item['level1'] = level1.index(label_two)
                if len(label) > 2:
                    if label_two in level2_pass:
                        item['level2'] = level2.index("None")
                    else:
                        if label[2] not in level2:
                            print(label[2])
                            item['level2'] = level2.index("None")
                        else:
                            item['level2'] = level2.index(label[2])
                else:
                    item['level2'] = level2.index("None")
        except:
            print(label, item['text'])
            raise
            
    final_data = {"id": None, "text": None, "level1": None, "level2": None}
    final_data['id'] = torch.tensor([item['id'] for item in data])
    final_data['text'] = [item['text'] for item in data]
    final_data['level1'] = torch.tensor([item['level1'] for item in data])
    final_data['level2'] = torch.tensor([item['level2'] for item in data])
    try:
        x = tokenizer.batch_encode_plus(final_data['text'], return_tensors="pt", padding="longest", max_length=512, truncation=True)
    except:
        print(final_data['text'])
        raise
    for key in x:
        final_data[key] = x[key]
        
    return final_data

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class NewsRepModule(nn.Module):
    def __init__(self, roberta):
        super().__init__()
        self.roberta = roberta
        self.drop_layer = nn.Dropout(p=0.6)

    def forward(self, input_ids, attention_mask):
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.pooler_output
        logits = self.drop_layer(logits)
        return logits

class Level2Calssification(nn.Module):
    def __init__(self,):
        super().__init__()
        self.classification = nn.Linear(768, 8)

    def forward(self, logits):
        _out = self.classification(logits)
        return _out
    
class Level1Calssification(nn.Module):
    def __init__(self,):
        super().__init__()
        self.classification = nn.Linear(768, 50)
        self.relu = nn.LeakyReLU()
        self.classification2 = nn.Linear(50, 16)
        

    def forward(self, logits):
        _out = self.classification(logits)
        _out = self.relu(_out)
        _out = self.classification2(_out)
        return _out

In [11]:
for item in loader:
    pass

In [11]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from transformers import AutoTokenizer, RobertaModel, BertModel
import json

level1 = [
    "comp.os", "comp.sys", "comp.windows", "comp.graphics", "rec.motorcycles", "rec.sport", "rec.autos", "talk.religion",
    "sci.electronics", "sci.med", "sci.space", "misc.forsale", "talk.politics", "sci.crypt", "alt.atheism", "soc.religion"
]
level2 = [
    "misc", "guns", "ibm", "mac", "baseball", "hockey", "mideast", "None"
]

best_performances = []
for split in range(5):
    loader = DataLoader(new_datasets[split]['train'], batch_size=36, collate_fn=collate_label_set)
    val_loader = DataLoader(new_datasets[split]['validation'], batch_size=36, collate_fn=collate_label_set)
    device = "cuda:6"
    rep_model = NewsRepModule(BertModel.from_pretrained("bert-base-uncased"))
    rep_model = rep_model.to(device)

    level1_classification = Level1Calssification().to(device)
    level2_classification = Level2Calssification().to(device)
    
    params = list(rep_model.parameters()) + list(level1_classification.parameters())
    params = list(params) + list(level2_classification.parameters())
    optim = torch.optim.AdamW(params, lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    
    best_performance = 0
    best_performance_details = None
    
    for epoch in range(5):
        rep_model.train()
        level1_classification.train()
        level2_classification.train()
        total_loss = 0
        for train_batch in tqdm(loader):
            try:
                rep = rep_model(train_batch['input_ids'].to(device),
                                               train_batch['attention_mask'].to(device))
                level1 = level1_classification(rep)
                level2 = level2_classification(rep)

            except:
                print(train_batch['text'])
                raise
            try:
                loss = 0 
                loss += criterion(level1, train_batch['level1'].long().to(device))
                loss += criterion(level2, train_batch['level2'].long().to(device))
                total_loss += loss.item()
                loss.backward()
            except:
                print(level1, train_batch['level1'])
                print(level2, train_batch['level2'])
                raise
            optim.step()
            optim.zero_grad()
        print(f"The loss of epoch {epoch} is {total_loss} \n")

        rep_model.eval()
        level1_classification.eval()
        level2_classification.eval()
        all_preds = {'level1': [], 'level2': []}
        all_gt = {'level1': [], 'level2': []}
        
        for train_batch in tqdm(val_loader):
            rep = rep_model(train_batch['input_ids'].to(device),
                                                   train_batch['attention_mask'].to(device))
            level1 = level1_classification(rep)
            level2 = level2_classification(rep)
            level1 = level1.argmax(-1)
            level2 = level2.argmax(-1)
            all_gt['level1'].extend(train_batch['level1'].cpu().detach().tolist())
            all_gt['level2'].extend(train_batch['level2'].cpu().detach().tolist())
            all_preds['level1'].extend(level1.cpu().detach().tolist())
            all_preds['level2'].extend(level2.cpu().detach().tolist())

            
        l1_pr, l1_rec, l1_f1, l1_sup = precision_recall_fscore_support(np.array(all_gt['level1']), np.array(all_preds['level1']))
        l2_pr, l2_rec, l2_f1, l2_sup = precision_recall_fscore_support(np.array(all_gt['level2']), np.array(all_preds['level2']))
        level1_aggregate = (torch.from_numpy(l1_sup) * torch.from_numpy(l1_f1)).sum().item() / torch.from_numpy(l1_sup).sum().item()
        level2_aggregate = (torch.from_numpy(l2_sup) * torch.from_numpy(l2_f1)).sum().item() / torch.from_numpy(l2_sup).sum().item()
        score = (level1_aggregate + level2_aggregate) / 2
        print(score, level1_aggregate, level2_aggregate)
        if score > best_performance:
            best_performance = score
            best_performance_details = [torch.from_numpy(l1_pr), torch.from_numpy(l1_rec), 
                                        torch.from_numpy(l1_f1), torch.from_numpy(l1_sup),
                                        torch.from_numpy(l2_pr), torch.from_numpy(l2_rec),
                                        torch.from_numpy(l2_f1), torch.from_numpy(l2_sup),
                                       level1_aggregate, level2_aggregate]
    if len(best_performances):
        for _ind in range(len(best_performances)):
            best_performances[_ind] = best_performances[_ind] + best_performance_details[_ind]
    else:
        best_performances = best_performance_details

for _ind in range(len(best_performances)):
    best_performances[_ind] = best_performances[_ind] / 5

torch.save(best_performances, "average_performance.pt")

100%|██████████| 237/237 [03:43<00:00,  1.06it/s]


The loss of epoch 0 is 810.6471781730652 



100%|██████████| 60/60 [00:26<00:00,  2.25it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6483379718945594 0.44093008214555257 0.8557458616435663


100%|██████████| 237/237 [03:43<00:00,  1.06it/s]


...

100%|██████████| 60/60 [00:27<00:00,  2.15it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.9069513812953915 0.8349145429187443 0.9789882196720389


 86%|████████▋ | 205/237 [03:13<00:30,  1.06it/s]

tensor([[-3.1823e-01, -1.0949e+00,  2.9482e-01,  1.2644e-01,  2.8600e+00,
          6.1529e-01,  7.1357e-01,  1.0294e-01,  9.4741e-01,  2.3661e-01,
          7.9043e-01, -8.7302e-01, -4.1094e-01,  1.1247e+00, -1.3507e+00,
         -5.4911e-01],
        [-1.5400e+00, -9.1800e-01, -1.1939e-01,  2.3895e-01,  6.3017e-01,
         -9.5284e-01, -1.2238e-01,  2.8385e-01,  9.2992e-02,  6.6009e-01,
          3.5780e+00, -6.2653e-03, -1.7570e+00,  3.1946e-02,  1.8585e-01,
          3.1362e-01],
        [-3.7071e-01, -1.8258e+00,  5.1810e-01,  1.6185e+00,  8.3523e-02,
         -2.9867e-01,  1.0451e-01,  3.1893e-01,  2.3528e+00,  3.5191e-01,
          3.4971e-01, -7.1055e-01,  4.7012e+00,  5.2888e-01, -4.9854e-02,
         -5.1418e-01],
        [ 1.7748e+00, -2.0178e+00,  7.5961e-02,  3.9628e+00,  8.1556e-01,
          1.3460e+00,  1.2599e-01,  3.2697e-01,  2.6539e-01, -6.6937e-01,
          5.9200e-01,  1.9980e+00,  3.0663e-01, -2.0400e+00, -3.5470e-01,
         -8.0112e-01],
        [ 1.1856e+00

KeyboardInterrupt: 

In [11]:
torch.save(rep_model.state_dict(), "rep_model2_base.pt")
torch.save(level1_classification.state_dict(), "level1_classification2_base.pt")
torch.save(level2_classification.state_dict(), "level2_classification2_base.pt")


In [10]:
level1_classification_path = "level1_classification2_base.pt"
level1_classification.load_state_dict(torch.load(level1_classification_path, map_location=device))

level2_classification_path = "level2_classification2_base.pt"
level2_classification.load_state_dict(torch.load(level2_classification_path, map_location=device))

rep_model_path = "rep_model2_base.pt"
rep_model.load_state_dict(torch.load(rep_model_path, map_location=device))
    

<All keys matched successfully>

In [13]:
from tqdm import tqdm 

rep_model.eval()
level1_classification.eval()
level2_classification.eval()
for dataload in [loader, test_loader]:
    level1_correct, level2_correct = 0, 0
    level1_correct_actual, level2_correct_actual = 0, 0
    level1_total, level2_total = 0, 0
    level1_total_actual, level2_total_actual = 0, 0
    for train_batch in tqdm(dataload):
        rep = rep_model(train_batch['input_ids'].to(device),
                                               train_batch['attention_mask'].to(device))
        level1 = level1_classification(rep)
        level2 = level2_classification(rep)
        level1 = level1.argmax(-1)
        level2 = level2.argmax(-1)

        level1_correct += (level1 == train_batch['level1'].long().to(device)).sum().item()
        level1_total += train_batch['level1'].shape[0]
        level1_correct_actual += ((level1 == train_batch['level1'].long().to(device)) & (train_batch['level1'].long().to(device) != 16)).sum().item()
        level1_total_actual += (train_batch['level1'].long().to(device) != 16).sum().item()

        level2_correct += (level2 == train_batch['level2'].long().to(device)).sum().item()
        level2_total += train_batch['level2'].shape[0]
        level2_correct_actual += ((level2 == train_batch['level2'].long().to(device)) & (train_batch['level2'].long().to(device) != 7)).sum().item()
        level2_total_actual += (train_batch['level2'].long().to(device) != 7).sum().item()

    print(level1_correct/level1_total)
    print(level2_correct/level2_total)
    print(level1_correct_actual/level1_total_actual)
    print(level2_correct_actual/level2_total_actual)

100%|██████████| 297/297 [02:10<00:00,  2.27it/s]


0.8480870217554388
0.9652100525131283
0.8480870217554388
0.9168021680216802


100%|██████████| 195/195 [01:23<00:00,  2.33it/s]

0.754096025074797
0.8704943724177233
0.754096025074797
0.7293783015034538


In [3]:
import torch
performance = torch.load("average_performance.pt")

In [4]:
performance

[tensor([0.6895, 0.7998, 0.8308, 0.7274, 0.7668, 0.9598, 0.7902, 0.1177, 0.7301,
         0.8964, 0.7986, 0.8041, 0.8223, 0.8103, 0.5584, 0.6624],
        dtype=torch.float64),
 tensor([0.6919, 0.8255, 0.8420, 0.6733, 0.7495, 0.9485, 0.8089, 0.0383, 0.6501,
         0.8724, 0.8599, 0.7947, 0.9044, 0.8152, 0.5500, 0.8215],
        dtype=torch.float64),
 tensor([0.6810, 0.8102, 0.8345, 0.6973, 0.7577, 0.9541, 0.7990, 0.0475, 0.6869,
         0.8835, 0.8266, 0.7986, 0.8609, 0.8114, 0.5521, 0.7321],
        dtype=torch.float64),
 tensor([111.0000, 219.8000, 115.2000, 110.2000, 110.2000, 221.2000, 107.8000,
          67.8000, 112.6000, 114.4000, 112.8000, 113.0000, 297.0000, 113.6000,
          89.8000, 116.4000]),
 tensor([0.7223, 0.7400, 0.6812, 0.8323, 0.9566, 0.9504, 0.8921, 0.9351],
        dtype=torch.float64),
 tensor([0.7375, 0.7908, 0.6603, 0.7413, 0.9141, 0.9249, 0.8714, 0.9439],
        dtype=torch.float64),
 tensor([0.7291, 0.7614, 0.6675, 0.7811, 0.9346, 0.9374, 0.8810, 0.9394]

In [9]:
(performance[-3] * performance[-4])[:-1].sum() / performance[-3][:-1].sum()

tensor(0.8153, dtype=torch.float64)

In [11]:
(performance[-7] * performance[-8]).sum() / performance[-7].sum()

tensor(0.7760, dtype=torch.float64)